# API Gateway Demo

## Create Job

In [1]:
import json

import requests

def create_dummy_jobs(api_url: str, job_name: str, work_count: int):

    # Construct the full URL
    api_url = api_url.rstrip('/')
    endpoint = f"{api_url}/create_job"

    # Headers
    headers = {"Content-Type": "application/json"}

    works = [
        {
            "work_id": f"work_{i}",
            "s3_uris": ["s3://bucket/file1.txt"]
        }
        for i in range(work_count)
    ]    
    request_body = {"job_name": job_name, "works": works}

    try:
        # Make the POST request
        response = requests.post(endpoint, data=json.dumps(request_body), headers=headers)
        
        # Check if the request was successful
        if response.status_code == 200:
            # Parse the JSON response
            data = response.json()
            print("API Response:", data)
        else:
            print(f"Error: API request failed with status code {response.status_code}")
            print("Response:", response.text)

    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")

create_dummy_jobs(
    api_url="https://1orpzlyvla.execute-api.us-east-1.amazonaws.com/dev/",
    job_name="job_001",
    work_count=10,
)


API Response: {'message': 'Success'}


## Check Job Progress

In [4]:
import logging

import requests

def get_job_progress(api_url: str, job_name: str):
    """
    Query the job_progress endpoint with the given job_name.

    Args:
    api_url (str): The base URL of your API Gateway
    job_name (str): The name of the job to query

    Returns:
    dict: The JSON response from the API, or None if an error occurred
    """
    api_url = api_url.rstrip('/')
    # Construct the full URL
    endpoint = f"{api_url}/job_progress"
    
    # Set up the query parameters
    params = {'job_name': job_name}
    
    try:
        # Make the GET request
        response = requests.get(endpoint, params=params)
        
        # Check the status code
        if response.status_code == 200:
            return response.json()
        elif response.status_code == 404:
            logging.info(f"No data found for job_name: {job_name}")
            return response.json()
        else:
            response.raise_for_status()
    
    except requests.RequestException as e:
        logging.exception(f"An error occurred: {e}")


get_job_progress(
    api_url="https://1orpzlyvla.execute-api.us-east-1.amazonaws.com/dev/",
    job_name="job_001",
)

{'IN_QUEUE': ['work_0',
  'work_1',
  'work_2',
  'work_3',
  'work_4',
  'work_5',
  'work_6',
  'work_7',
  'work_8',
  'work_9']}